In [ ]:
!pwd
!ls

/content
cennik.csv  sample_data


In [ ]:
!pip install gradio

In [ ]:
!pip install llama-index-llms-llama-cpp==0.3.0
!pip install llama-index-llms-groq
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install llama-index-readers-file

In [ ]:
from llama_index.llms.groq import Groq
from getpass import getpass
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import Settings
from os import sep
from llama_index.readers.file import PandasCSVReader
from llama_index.core import Document
import pandas as pd
from llama_index.core.schema import TextNode
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from getpass import getpass
import gradio as gr

In [ ]:
GROQ_API_KEY = getpass()

··········


In [ ]:
llm = Groq(model="llama3-8b-8192", api_key=GROQ_API_KEY)

In [ ]:
data = pd.read_csv("./cennik.csv", delimiter=";", on_bad_lines="warn")

documents = []
nodes = []
for _, row in data.iterrows():
    metadata = {
        "NAZWA": row["NAZWA"],
        "KATEGORIE": row["KATEGORIE"],
        "PRODUCENT": row["PRODUCENT"],
        "CENA_KATALOGOWA": row["CENA_KATALOGOWA"],
        "CENA_KLIENT": row["CENA_KLIENT"],
        "WALUTA": row["WALUTA"],
        "VAT": row["VAT"],
        "RABAT_KLIENTA": row["RABAT_KLIENTA"],
        "WAGA": row["WAGA"],
        "OPAKOWANIE": row["OPAKOWANIE"],
        "JEDNOSTKA_MIARY": row["JEDNOSTKA_MIARY"],
        "TYP_KURIERA": row["TYP_KURIERA"],
        "KURIERZY_NAZWA": row["KURIERZY_NAZWA"],
        "ACTIVE": row["ACTIVE"],
    }

    # Treść węzła: nazwa i opis produktu
    content = f"{row['NAZWA']}, czyli {row['OPIS']}"
    nodes.append(TextNode(text=content, metadata=metadata))


In [ ]:
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-small-en-v1.5")
index=VectorStoreIndex(nodes, embed_model=embed_model)
Settings.embed_model = embed_model

# dla dokumentów:
# index=VectorStoreIndex.from_documents(documents, embed_model=embed_model)

In [ ]:
query_engine=index.as_query_engine(llm=llm)

In [ ]:

response = query_engine.query("Jaki jest najtańszy produkt? Jaką ma nazwę i ile kosztuje?")
print(response)

The cheapest product is NAKLEJKA NA MUCHY 2SZT, which costs 6.55 PLN.


In [ ]:
response = query_engine.query("Czy mogę w sklepie zaopatrzyć się w akcesoria do remontu łazienki?")
print(response)

Tak, w sklepie dostępne są różne akcesoria, które mogą pomóc w remontowaniu łazienki.


In [ ]:
response = query_engine.query("W jakie akcesoria do remontu łazienki mogę zaopatrzeć się w sklepie i jaka jest ich cena?")
print(response)

You can equip yourself with a scraper with a replaceable blade, which is suitable for removing paint, stickers, and other residues from glass, doors, and tiles, for a price of 12.99 PLN. Additionally, you can consider a handle with a removable blade, which is suitable for various tasks, for a price of 76.42 PLN.


In [ ]:
response = query_engine.query("W jakiej cenie jest skrobak z wymiennym ostrzem?")
print(response)

12,99 PLN


In [ ]:
def chatbot(input_text):
    query_engine = index.as_query_engine(llm=llm)
    response = query_engine.query(input_text)

    return response

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://467b7162d3fe668a51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
